# Pro-Test Model Evaluation & Comparison

This notebook demonstrates the model training, evaluation, and comparison workflow for Pro-Test v2.0.

## Contents
1. Data Loading & Exploration
2. Model Comparison (RF, XGBoost, LightGBM)
3. Ensemble Training
4. Feature Importance Analysis
5. Model Registration

In [ ]:
# Imports
import sys

sys.path.insert(0, "..")

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from protest.models import (
    EnsembleConfig,
    EnsembleModel,
    ModelConfig,
    ModelRegistry,
    ModelType,
    compare_models,
    print_comparison_report,
)

# Set style
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("husl")

print("Imports complete!")

## 1. Data Loading & Exploration

In [ ]:
# Load data
df = pd.read_csv("../data/full_df.csv")
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Define features and targets
FEATURE_COLUMNS = [
    "country",
    "governorate",
    "locationtypeend",
    "demandtypeone",
    "tacticprimary",
    "violence",
]

TARGET_COLUMNS = [
    "teargas",
    "rubberbullets",
    "liveammo",
    "sticks",
    "surround",
    "cleararea",
    "policerepress",
]

# Handle combined_sizes
if "combined_sizes" not in df.columns:
    df["sizeestimate"] = df["sizeestimate"].fillna(-99)
    df["sizeexact"] = df["sizeexact"].fillna(0)
    df["combined_sizes"] = df["sizeexact"] + df["sizeestimate"]

# Replace unknown sizes with median
median_size = df.loc[df["combined_sizes"] > 0, "combined_sizes"].median()
df.loc[df["combined_sizes"] <= 0, "combined_sizes"] = median_size

FEATURE_COLUMNS.append("combined_sizes")

# Filter available columns
available_features = [c for c in FEATURE_COLUMNS if c in df.columns]
available_targets = [c for c in TARGET_COLUMNS if c in df.columns]

print(f"Features: {available_features}")
print(f"Targets: {available_targets}")

In [ ]:
# Prepare X and y
X = df[available_features].copy()
y = df[available_targets].copy()

# Convert targets to binary
for col in y.columns:
    y[col] = (y[col] > 0).astype(int)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print("\nTarget distribution:")
y.mean()

In [ ]:
# Visualize target distribution
fig, ax = plt.subplots(figsize=(10, 5))
y.mean().plot(kind="bar", ax=ax, color="steelblue")
ax.set_title("Target Variable Distribution (Positive Rate)")
ax.set_ylabel("Proportion")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 2. Model Comparison

In [ ]:
# Compare all model types
results = compare_models(
    X,
    y,
    model_types=[ModelType.RANDOM_FOREST, ModelType.XGBOOST, ModelType.LIGHTGBM],
    n_folds=5,
)

# Print comparison report
report = print_comparison_report(results)
print(report)

In [ ]:
# Visualize comparison
metrics_df = pd.DataFrame(
    [
        {
            "Model": r.model_type.value,
            "Accuracy": r.overall_metrics.accuracy,
            "Precision": r.overall_metrics.precision,
            "Recall": r.overall_metrics.recall,
            "F1": r.overall_metrics.f1,
        }
        for r in results
    ]
)

fig, ax = plt.subplots(figsize=(12, 5))
metrics_df.set_index("Model")[["Accuracy", "Precision", "Recall", "F1"]].plot(kind="bar", ax=ax)
ax.set_title("Model Comparison - Overall Metrics")
ax.set_ylabel("Score")
ax.set_ylim(0, 1)
ax.legend(loc="lower right")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 3. Ensemble Training

In [ ]:
# Train ensemble model
config = ModelConfig(
    target_columns=available_targets,
    feature_columns=available_features,
)

ensemble_config = EnsembleConfig(
    model_types=[ModelType.RANDOM_FOREST, ModelType.XGBOOST, ModelType.LIGHTGBM],
    weights=[0.4, 0.35, 0.25],  # Weighted based on comparison results
    voting="soft",
)

ensemble = EnsembleModel(config=config, ensemble_config=ensemble_config)
ensemble.fit(X, y)

print("Ensemble trained!")
print(f"Model ID: {ensemble.metadata.model_id}")

In [ ]:
# Test predictions with confidence
sample = X.iloc[:5]
mean_proba, std_proba = ensemble.predict_proba_with_confidence(sample)

print("Sample predictions with confidence intervals:")
for i, target in enumerate(available_targets):
    if i < len(mean_proba):
        prob = mean_proba[i][0, 1]  # Probability of positive class
        std = std_proba[i][0, 1]  # Standard deviation
        print(f"  {target}: {prob:.3f} ± {std:.3f}")

## 4. Feature Importance Analysis

In [ ]:
# Get feature importance
importance = ensemble.get_feature_importance()

# Plot top 20 features
top_features = dict(list(importance.items())[:20])

fig, ax = plt.subplots(figsize=(10, 8))
pd.Series(top_features).plot(kind="barh", ax=ax, color="steelblue")
ax.set_title("Top 20 Feature Importances (Ensemble)")
ax.set_xlabel("Importance")
plt.tight_layout()
plt.show()

## 5. Model Registration

In [ ]:
# Register model
registry = ModelRegistry("../models/registry")

version = registry.register_model(
    ensemble,
    name="protest_predictor",
    description="Ensemble model (RF+XGB+LGBM) for protest outcome prediction",
    tags={"type": "ensemble", "targets": ",".join(available_targets)},
)

print(f"Registered model version: {version.version}")
print(f"Artifact path: {version.artifact_path}")

In [ ]:
# List registered models
models = registry.list_models()
for name, versions in models.items():
    print(f"\n{name}:")
    for v in versions:
        print(f"  - {v.version} ({v.stage}) - {v.created_at}")

In [ ]:
# Promote to production
registry.promote_model("protest_predictor", version.version, "production")
print(f"Promoted {version.version} to production!")

## Summary

This notebook demonstrated:
1. Loading and preparing protest event data
2. Comparing Random Forest, XGBoost, and LightGBM models
3. Training a weighted ensemble model
4. Getting predictions with confidence intervals
5. Analyzing feature importance
6. Registering and promoting models

The ensemble model combines the strengths of all three algorithms for more robust predictions.